# Prepayments by Month and Servicer

Barclays does this monthly Prepayment report where they talk about trends in prepayments. 

Steve:
"I am looking to expand on the outlook to talk more about the factor drivers, what I have is fairly weak right now.  This will be used in our investment process as well."

> **Outlook**
> Current prepayment speeds for both the 3.5% and 3.0% coupon have marginally slowed from the three and six month average CPRs of 17.8% and 20.1% for the 3.5% coupon and 10.5% and 12.6% for the 3.0% coupon. We expect prepayments to remain low as the as the January MIP cut continues to reverberate through the market. We expect a slight increase in speeds from the February 3.5 and 3.0 coupon index levels of 12.2% and 3.0% CPR as government refinance activity picked up 6.5% over the month as rates were relatively unchanged.

I need to look at recent prepayment activity in GNM IIs, and chop it up by coupon and vintage, possibly other things.

In [1]:
import os
os.chdir("../")
import prepayments as pp
import ggplot as gg
from time import mktime
import pandas as pd
import cPickle as pickle

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
gnm_data = pp.gnm_data.GNM_Pool()

# Prepayments by Month

For each pool, calculate monthly CPR and preserve coupon and vintage info.

In [6]:
gnm2_cusips = list(set(gnm_data.h5file.root.candidate_pools.col("cusip")))

In [7]:
len(gnm2_cusips)

45167

Only cusips that were active since, say, last September.

In [8]:
recent_gnm2_cusips = []
for cusip in gnm2_cusips:
    p_data = gnm_data.df_for_cusip(cusip, trim=False)
    if p_data.ix[datetime.date(2016,9,1):].shape[0] > 0:
        recent_gnm2_cusips.append(cusip)

In [9]:
with open("/tmp/recent_gnm2_cusips.pkl","wb") as f:
    pickle.dump(recent_gnm2_cusips, f)

In [7]:
cprs = pd.DataFrame({cusip:gnm_data.cpr_for_cusip(cusip)['2016-09-01':] for cusip in recent_gnm2_cusips})

prepayments/gnm_data.py:477: RuntimeWarning: divide by zero encountered in double_scalars
  principal_payment = rate / ((1+rate)**(n_payments - loan_age)-1)


In [8]:
cprs.shape

(6, 39269)

In [9]:
cprs.index

DatetimeIndex(['2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01'],
              dtype='datetime64[ns]', freq='MS')

No March data!?!?!

In [10]:
march_gnm2_cusips = []
for cusip in gnm2_cusips:
    p_data = gnm_data.df_for_cusip(cusip, trim=False)
    if p_data.ix[datetime.date(2017,3,1):].shape[0] > 0:
        march_gnm2_cusips.append(cusip)

In [11]:
len(march_gnm2_cusips)

0

So there's no prepayment data available for March yet, for GNM IIs, at least.

Let's rerun the model with some samples for January so that we have an estimate for vintage betas. 

In [15]:
cprs[recent_gnm2_cusips[150]]

2016-09-01    0.248806
2016-10-01   -0.000215
2016-11-01    0.246030
2016-12-01    0.005445
2017-01-01   -0.001481
2017-02-01    0.242500
Freq: MS, Name: 3617A32R0, dtype: float64

... but! BBG CPR is forward a month to CPR calculated from GNM data, so that 0.2425 for cusip `3617A32R0` shows up as March data if you do `G2 A03483 Mtg PDI PERF`. So I have samples for January and February.

Let's look at prepayments by issuer, vintage, and coupon.

In [27]:
p_data.ix[-1,['cusip','issuer_name','security_interest_rate','pool_issue_date']]

cusip                                                    3617A32R0
issuer_name               Pennymac Loan Services, Llc             
security_interest_rate                                        4000
pool_issue_date                                2015-07-01 00:00:00
Name: 2017-02-01, dtype: object

In [14]:
p_data.columns

Index([u'aols', u'as_of_date', u'cusip', u'few_units_number_of_loans',
       u'few_units_pct_of_upb', u'few_units_upb', u'first_quartile_aols_q1',
       u'first_quartile_cltv_q1', u'first_quartile_ltv_q1',
       u'first_quartile_wac_q1', u'first_quartile_wagm_q1',
       u'first_quartile_wala_q1', u'first_quartile_waolt_q1',
       u'first_quartile_warm_q1', u'hamp_modified_number_of_loans',
       u'hamp_modified_pct_of_total_upb', u'hamp_modified_upb',
       u'interest_adjustment_date', u'issuer_name', u'issuer_number',
       u'loan_purpose_not_available_number_of_loans',
       u'loan_purpose_not_available_pct_of_upb',
       u'loan_purpose_not_available_upb', u'look_back_period',
       u'ltv_not_available_number_of_loans', u'ltv_not_available_pct_of_upb',
       u'ltv_not_available_upb', u'maximum_aols_q4', u'maximum_cltv_q4',
       u'maximum_ltv_q4', u'maximum_wac_q4', u'maximum_wagm_q4',
       u'maximum_wala_q4', u'maximum_waolt_q4', u'maximum_warm_q4',
       u'median_ao

In [11]:
pool_info = list()
for cusip in recent_gnm2_cusips:
    p_data = gnm_data.df_for_cusip(cusip)
    pool_info.append(p_data.ix[-1,['cusip','issuer_name', 'wac', 'pool_upb',
                                   'security_interest_rate','pool_issue_date']])

In [12]:
pool_info = pd.DataFrame.from_records(pool_info)
pool_info.to_csv("/data/prepayments/gnm_ii_pool_info.csv")

In [36]:
cprs.to_csv("/data/prepayments/gnm_ii_pool_recent_cprs.csv")

# Get new samples

In [4]:
with open("/tmp/recent_gnm2_cusips.pkl","rb") as f:
    recent_gnm2_cusips = pickle.load(f)

In [5]:
pm = pp.PoolModel()

In [9]:
data = dict()
for cusip in recent_gnm2_cusips:
    data[cusip] = pm.current_month_data_for_cusip(cusip, dt=pd.Timestamp("2017-03-29"))

ERROR:root:Called current_month_data method for 36179QBR1, but it was paid off last month.
ERROR:root:Called current_month_data method for 36292HUY7, but it was paid off last month.
ERROR:root:Called current_month_data method for 36176BR82, but it was paid off last month.
ERROR:root:Called current_month_data method for 36294RGU7, but it was paid off last month.
ERROR:root:Called current_month_data method for 36294P3C5, but it was paid off last month.
ERROR:root:Called current_month_data method for 36209DP57, but it was paid off last month.
ERROR:root:Weighted Average Coupon is missing for 36202ESG5, 2017-03-29
ERROR:root:Called current_month_data method for 36292D7A4, but it was paid off last month.
ERROR:root:Called current_month_data method for 36202A2R7, but it was paid off last month.
ERROR:root:Called current_month_data method for 36296RMG9, but it was paid off last month.
ERROR:root:Called current_month_data on 36202DQ79, but the last data we have is 2016-10-01 00:00:00, more tha

In [10]:
p_data = pd.DataFrame.from_dict({k:v for k,v in data.items() if v is not None}, orient='index')

In [11]:
p_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38627 entries, 36176A5R6 to 36297KZ38
Data columns (total 14 columns):
upfront_mip               38627 non-null float64
lockin                    38627 non-null float64
wala                      38627 non-null int64
seasonality               38627 non-null int64
wac                       38627 non-null int64
origination_dt            38627 non-null datetime64[ns]
upb                       38627 non-null int64
incentive                 38627 non-null float64
cato                      38627 non-null float64
security_interest_rate    38627 non-null int64
burnout                   38627 non-null float64
dt                        38627 non-null datetime64[ns]
hpa                       38627 non-null float64
sato                      38627 non-null float64
dtypes: datetime64[ns](2), float64(7), int64(5)
memory usage: 4.4+ MB


In [12]:
p_data.reset_index().to_csv("/data/prepayments/201703 gnm_ii_data.csv")

Missing all Febs. Because I *am* taking CPRs at the correct month. Bloomberg sucks.

In [20]:
cprs = pd.read_csv("/data/prepayments/gnm_ii_pool_recent_cprs.csv")


In [25]:
data_yc = pp.bloomberg_data.YC_Data(read_only=False)

ValueError: The file 'data/bbg_data.h5' is already opened, but in read-only mode.  Please close it before reopening in append mode.